# Hyperparameter Tuning using HyperDrive
Import dependencies

In [1]:
import os
import json
import azureml.core
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core import Environment, ScriptRunConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

print("SDK version:", azureml.core.VERSION)

SDK version: 1.42.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hdr-heart-experiment'

experiment=Experiment(ws, experiment_name)
experiment.start_logging()

Experiment,Id,Type,Status,Details Page,Docs Page
hdr-heart-experiment,4b234ee0-c6c4-456d-a922-f61cc426a363,,Running,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

In [7]:
key = "heart-failure"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [8]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Hyperdrive Configuration

I used the Logistic Regression algorithm from sklearn.
* **C-** The inverse regularization strength
* **max_iter-** The maximum iteration to converge 

I used random parameter sampling to sample over a discrete set of values. Random parameter sampling is great for discovery and getting hyperparameter combinations that you would not have guessed intuitively, although it requires more time to execute.<br>
Parameters for search space C is [0.01, 0.1, 1, 10, 100] and for max_iter is [20, 50, 100, 120, 150]<br>
The BanditPolicy is based on slack factor/ slack amount and evaluation interval. Bandit terminates runs where the primary metric is not within the specified slack factor/slack amount compared to the best performing run. This helps to improves computational efficiency.

In [10]:
# Create an early termination policy. 
early_termination_policy =BanditPolicy(
    slack_factor = 0.1,
    evaluation_interval= 2,
    delay_evaluation= 10,
)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1.0, 10.0, 100.0),
        '--max_iter': choice(20,50, 100, 120, 150)
    }
)
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_env.yml')

# Create your estimator and hyperdrive config
src =  ScriptRunConfig(
    source_directory= "./",
    script = "train.py",
    compute_target = compute_target,
    environment=sklearn_env
)


hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling = param_sampling,
    primary_metric_name= 'Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4

)


In [11]:
# Submit your experiment
hdr = experiment.submit(
    config = hyperdrive_config
)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?



In [12]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
hdr.wait_for_completion(show_output=True)

RunId: HD_23574a49-7658-4a2c-a8ef-760118d6cfe7
Web View: https://ml.azure.com/runs/HD_23574a49-7658-4a2c-a8ef-760118d6cfe7?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-202911/workspaces/quick-starts-ws-202911&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-08-01T16:06:27.144292][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-08-01T16:06:27.8501053Z][SCHEDULER][INFO]Scheduling job, id='HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_0' 
[2022-08-01T16:06:27.9815906Z][SCHEDULER][INFO]Scheduling job, id='HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_1' 
[2022-08-01T16:06:28.1226552Z][SCHEDULER][INFO]Scheduling job, id='HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_2' 
[2022-08-01T16:06:28.2543042Z][SCHEDULER][INFO]Scheduling job, id='HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_3' 
[2022-08-01T16:06:28.3833514Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_0

{'runId': 'HD_23574a49-7658-4a2c-a8ef-760118d6cfe7',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2022-08-01T16:06:26.411872Z',
 'endTimeUtc': '2022-08-01T16:14:31.865517Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '31dd6fb2-0f2d-489a-8627-49410eba9697',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1083-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.42.0',
  'space_size': '25',
  'score': '0.8133333333333334',
  'best_child_run_id': 'HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_11',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_11'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientTyp

## Best Model


In [14]:
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

In [15]:
print("Best Run Id : {}".format(best_run.id), 
      "Accuracy : {}".format(best_run_metrics['Accuracy']), 
      "Best metrics : {}".format(best_run_metrics), sep = '\n')

Best Run Id : HD_23574a49-7658-4a2c-a8ef-760118d6cfe7_11
Accuracy : 0.8133333333333334
Best metrics : {'Regularization Strength:': 1.0, 'Max iterations:': 50, 'Accuracy': 0.8133333333333334}


In [16]:
model = best_run.register_model(
    model_name = 'heart-failure-best-model-hdr',
    model_path = './'
)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.